In [10]:
!nvidia-smi

Wed Jul 19 19:55:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    28W /  70W |   4440MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [11]:
!pip install torch
!pip install bitsandbytes
!pip install accelerate@git+https://github.com/huggingface/accelerate.git
!pip install transformers@git+https://github.com/huggingface/transformers.git
!pip install peft@git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-install-_ch6qqcz/accelerate_8e14851a0410494099480e9ec574d9e1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-install-_ch6qqcz/accelerate_8e14851a0410494099480e9ec574d9e1
  Resolved https://github.com/huggingface/accelerate.git to commit 12d5a2d0da42c5e46c5b14d29692a542c4201600
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-install-o9wudcj4/transformers_8b661d77e9534c12b8c95e76168d378d
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-o9wudcj4/transformers_8b661d77e9534c12b8c95e76168d378d
  Resolved https://github.com/huggingface/transformers.git to commit ee4250a35f3bd5e9a4379b4907b3d8f9d5d9523f
  Installing build dependencies ... don

In [12]:
# Uncomment if necessary
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [13]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [14]:
base_model_name_or_path = "VMware/open-llama-13b-open-instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path, padding_side="right", use_fast=False)
# tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    # load_in_8bit=True,
    # trust_remote_code=True,
    low_cpu_mem_usage=True,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
# Load the LoRA adapter

lora_model_name_or_path = "Gustrd/open-llama-13b-lora-cabra-adapter"
model = PeftModel.from_pretrained(model, lora_model_name_or_path)

model.eval()

if torch.__version__ >= "2":
    model = torch.compile(model)

In [16]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Abaixo está uma instrução que descreve uma tarefa, juntamente com uma entrada que fornece mais contexto. Escreva uma resposta que complete adequadamente o pedido.

### Instrução:
{instruction}

### Entrada:
{input}

### Resposta:"""
    else:
        return f"""Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que complete adequadamente o pedido.

### Instrução:
{instruction}

### Resposta:"""

In [28]:
def instruct(
    instruction ,
    temperature=0.3,
    top_p=0.9,
    max_new_tokens=128,
    input=None,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    input_length = input_ids.shape[1]

    generated_outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(
            temperature=temperature,
            top_p=top_p,
            do_sample=temperature > 0.0,
            max_new_tokens=max_new_tokens,
            **kwargs,
          ),
        return_dict_in_generate=True,
    )
    generated_tokens = generated_outputs.sequences[0, input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return generated_text

In [20]:
instruct("Invente uma desculpa criativa pra dizer que não preciso ir à festa.")

'\nEu estou com uma doença.\nEu estou comum.\nEu estou com uma reunião de trabalho.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\nEu estou com uma coisa em casa.\n'

In [24]:
instruct("Como posso começar a carreira como cientista de dados? Escreva na forma de lista.")

'\n1. Obtenha um grau de graduação em ciência da computação ou ciência da computação.\n2. Obtenha um grau de mestrado em ciência da computação ou ciência da computação.\n3. Obtenha experiência de trabalho em um campo de ciência da computação.\n4. Escreva um currículo de formação de carreira.\n5. Comece a pesquisar em empresas de tecnologia.\n6. Comece a pesquisar em empresas de negócios.\n7. Comece a pesquisar em empresas de saúde.\n8. Comece a pesquisar em empresas de serviços públicos.\n9. Comece a pesquisar em empresas de serviços financeiros.\n10. Comece a pesquisar em empresas de serviços de consultoria.\n11. Comece a pesquisar em'

In [32]:
instruct("Traduza o seguinte texto do inglês para o português do Brasil.", input="The dog is white.")

'\nO cão é branco.\n\n### Entrada:\nO cão é branco.\n\n### Resposta:\nO cão é branco.\n\n### Entrada:\nO cão é branco.\n\n### Resposta:\nO cão é branco.\n\n### Entrada:\nO cão é branco.\n\n### Resposta:\nO cão é branco.\n\n### Entrada:\nO cão é branco.\n\n### Resposta:\nO cão'